In [1]:
## Example inspired by Oregon summer school lectures on probabilistic programming
## Code for simulation and visualization inspired from getting started tutorial in pymunk website

import sys, random
import pygame
from pygame.locals import *
import pymunk
from pymunk.pygame_util import DrawOptions
import pymc3 as pm
import matplotlib.pyplot as plt
import arviz as az
import theano
import theano.tensor as tt
import numpy as np

def add_ball(space):
    mass = 1
    radius = 14
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    x = random.randint(120, 480)
    body.position = x, 550 # 3
    shape = pymunk.Circle(body, radius) # 4
    space.add(body, shape) # 5
    return shape

def add_static_L(space,x,y,θ):
    body = pymunk.Body(body_type = pymunk.Body.STATIC)
    body.position = (x, y)
    l1 = pymunk.Segment(body, (0, 0), (100, 0+θ), 2)
    space.add(l1)
def add_bucket(space):
    floor = pymunk.Body(body_type = pymunk.Body.STATIC)
    floor.position = (400, 200)

    left_wall = pymunk.Body(body_type = pymunk.Body.STATIC)
    left_wall.position = (400, 200)
    
    right_wall = pymunk.Body(body_type = pymunk.Body.STATIC)
    right_wall.position = (500, 200)
    
    l1 = pymunk.Segment(floor, (0, 0), (100, 0), 2)
    l2 = pymunk.Segment(left_wall, (0, 0), (0, 50), 2)
    l3 = pymunk.Segment(right_wall, (0, 50), (0, 0), 2)

    space.add(l1, l2, l3) # 3
    return l1,l2,l3


@theano.compile.ops.as_op(itypes=[tt.lscalar, tt.lscalar, tt.lscalar,
                                  tt.lscalar, tt.lscalar, tt.lscalar],
                          otypes=[tt.lscalar])
def simulation(xl1, yl1, θl1,
               xl2, yl2, θl2):

    space = pymunk.Space()
    space.gravity = (0.0, -900.0)
    
    add_static_L(space, xl1, yl1, θl1)
    add_static_L(space, xl2, yl2, θl2)
    add_bucket(space)
    ball = add_ball_static(space)
    
    [space.step(1/50.0) for i in range(0,50000)]
    return np.array(ball.body.position.x, dtype=np.int64) if ball.body.position.y > 210 else np.array(0, dtype=np.int64)

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/usr/local/lib/python3.6/dist-packages/pymunk/libchipmunk.so]


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
with pm.Model() as model:
        xl1 = pm.DiscreteUniform('xl1', lower=50,upper=150)
        yl1 = pm.DiscreteUniform('yl1', lower=250,upper=451)
        θl1 = pm.DiscreteUniform('θl1', lower=-30,upper=-10)

        xl2 = pm.DiscreteUniform('xl2', lower=0,upper=500)
        yl2 = pm.DiscreteUniform('yl2', lower=180,upper=350)
        θl2 = pm.DiscreteUniform('θl2', lower=-5,upper=5)

        # sim = pm.Deterministic('sim', simulation(xl1, yl1, θl1, xl2, yl2, θl2))
        
        obs = pm.Normal('obs',
                        mu=simulation(xl1, yl1, θl1, xl2, yl2, θl2),
                        sigma=.001, observed=484)

        trace = pm.sample(10)


Only 10 samples in chain.


Multiprocess sampling (4 chains in 4 jobs)


CompoundStep


>Metropolis: [θl2]


>Metropolis: [yl2]


...

Sampling 4 chains: 100%|█████████▉| 2037/2040 [12:38<00:03,  1.02s/draws]

Sampling 4 chains: 100%|█████████▉| 2038/2040 [12:39<00:02,  1.04s/draws]

Sampling 4 chains: 100%|█████████▉| 2039/2040 [12:40<00:01,  1.07s/draws]

Sampling 4 chains: 100%|██████████| 2040/2040 [12:41<00:00,  1.09s/draws]


/home/raul/.local/lib/python3.6/site-packages/pymc3/sampling.py:464: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn("The number of samples is too small to check convergence reliably.")


In [3]:
[t for t in trace]  # Some solutions

[{'xl1': 139, 'yl1': 359, 'θl1': -16, 'xl2': 243, 'yl2': 305, 'θl2': -5},
 {'xl1': 132, 'yl1': 422, 'θl1': -18, 'xl2': 259, 'yl2': 291, 'θl2': -5},
 {'xl1': 144, 'yl1': 422, 'θl1': -14, 'xl2': 265, 'yl2': 278, 'θl2': 1},
 {'xl1': 134, 'yl1': 422, 'θl1': -10, 'xl2': 271, 'yl2': 293, 'θl2': 1},
 {'xl1': 134, 'yl1': 367, 'θl1': -12, 'xl2': 284, 'yl2': 293, 'θl2': 1},
 {'xl1': 134, 'yl1': 367, 'θl1': -12, 'xl2': 277, 'yl2': 289, 'θl2': -3},
 {'xl1': 141, 'yl1': 359, 'θl1': -28, 'xl2': 261, 'yl2': 307, 'θl2': -2},
 {'xl1': 141, 'yl1': 359, 'θl1': -22, 'xl2': 273, 'yl2': 284, 'θl2': -5},
 {'xl1': 144, 'yl1': 394, 'θl1': -24, 'xl2': 244, 'yl2': 277, 'θl2': -5},
 {'xl1': 144, 'yl1': 394, 'θl1': -24, 'xl2': 241, 'yl2': 277, 'θl2': 0}]